# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

#### For this Homework of creating the model which has capacity to recognize the phoneme occured in the frame, my model managed to get high score cutoff after running or resuming it 4 times. Where I was resuming from the best model that occured in previous run.

#### To run the code file, Just Restart and clear all cell's output then run all

# Libraries

In [1]:
!pip install torchsummaryX==1.1.0 wandb --quiet

In [2]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
''' If you are using colab, you can import google drive to save model checkpoints in a folder
    If you want to use it, uncomment the two lines below
'''
# from google.colab import drive
# drive.mount('/content/drive')

' If you are using colab, you can import google drive to save model checkpoints in a folder\n    If you want to use it, uncomment the two lines below\n'

In [4]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [5]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"angeizabayo","key":"6727c97821f5c0aae21046c561c0545d"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.5 MB/s eta 0:00:00

  Preparing metadata (setup.py) ... done



  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73249 sha256=e1780501f6291d11fff49d31eea3a23bb0b55155aa460777db155157e9eb7b73

  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee

Successfully built kaggle


  Attempting uninstall: kaggle

    Found existing installation: kaggle 1.6.17

    Uninstalling kaggle-1.6.17:

      Successfully uninstalled kaggle-1.6.17



In [6]:
ls -ln


total 808

---------- 1 0 0 826381 Sep 25 20:40 __notebook__.ipynb


In [7]:
ls -ln /kaggle/working/

total 808

---------- 1 0 0 826547 Sep 25 20:40 __notebook__.ipynb


In [8]:
# commands to download data from kaggle
!kaggle competitions download -c 11785-hw1p2-f24

# !unzip -qo /content/11785-hw1p2-f24.zip -d '/content'


100%|██████████████████████████████████████▉| 3.97G/3.98G [00:24<00:00, 229MB/s]

100%|███████████████████████████████████████| 3.98G/3.98G [00:24<00:00, 175MB/s]


In [10]:
!unzip -qo 11785-hw1p2-f24.zip -d '/kaggle/working'


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [11]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = os.path.join(root, partition, 'mfcc')
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(root, partition, 'transcript')

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis = 0)
        #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir, transcript_names[i])) 
            transcript  = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript
 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        
            # TODO: Cepstral Normalization
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs = np.concatenate(self.mfccs, axis=0)     # concatenate

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts, axis=0)   # concatenate
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        padding = np.zeros((self.context, self.mfccs.shape[1]))
        self.mfccs = np.vstack([padding, self.mfccs, padding]) # TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        map_phone = {phoneme: idx for idx, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([map_phone.get(phoneme, 0) for phoneme in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        
        frames = frames = self.mfccs[ind: ind + 2 * self.context + 1]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [12]:
class AudioTestDataset(torch.utils.data.Dataset):
    
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, root, context= 0, partition= 'test-clean'):
        
        self.context = context
        
        
        self.mfcc_dir = os.path.join(root, partition, 'mfcc')
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        
        self.mfccs = []
        
        for i in range(len(mfcc_names)):
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            
            
            mfcc = (mfcc - np.mean(mfcc, axis = 0)) / np.std(mfcc, axis = 0)
            
            self.mfccs.append(mfcc)
        
        self.mfccs = np.concatenate(self.mfccs, axis=0)
        padding = np.zeros((self.context, self.mfccs.shape[1]))   # introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.vstack([padding, self.mfccs, padding])
        
        self.length = len(self.mfccs) - 2 * self.context
        
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, ind):
        
        ind = ind + self.context
        frames = self.mfccs[ind - self.context: ind + self.context + 1]
        frames = frames.flatten()

        frames = torch.FloatTensor(frames)
        
        return frames
            
        

    

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

#### I adjusted the number of epochs based on the expected outcomes, initially running 30 epochs, followed by sessions of 15, 15, and 5 epochs respectively. Our team observed that as the sample size per batch increased, the model's performance improved, where I choose a batch size of 8192. Additionally, I increased the context to 35 to provide more comprehensive input data for each prediction and set the learning rate at 0.001 to balance the speed and stability of the model’s training convergence.

In [13]:
config = {
    'epochs'        : 5,
    'batch_size'    : 8192,
    'context'       : 35,
    'init_lr'       : 1e-3,
    'architecture'  : 'medium-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [14]:
#TODO: Create a dataset object using the AudioDataset class for the training data

root_dir = '/kaggle/working/11785-f24-hw1p2'


train_data = AudioDataset(
    root = root_dir,
    phonemes = PHONEMES,
    context = config['context'],
    partition = 'train-clean-100'
    
)

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(
    root = root_dir,
    phonemes = PHONEMES,
    context = config['context'],
    partition = 'dev-clean'
)

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(
    root =  root_dir,
    context = config['context'],
    partition = 'test-clean'
)

In [15]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  8192

Context        :  35

Input size     :  1988

Output symbols :  42

Train dataset samples = 36091157, batches = 4406

Validation dataset samples = 1928204, batches = 236

Test dataset samples = 1934138, batches = 237


In [16]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([8192, 1988]) torch.Size([8192])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

#### I experimented with various network architectures, including Pyramid and Inverse Pyramid structures, but none met high cutoff or some was overfitting. Therefore, I utilized the mixed architecture GELU(ReLU was not working well with my network) as the activation function to introduce non-linearity effectively. At each layer I added a 0.3 dropout rate and batch normalization to combat overfitting and enhance generalization. Additionally, I implemented Xavier weight initialization to Ensure stable convergence during training. I managed to increase the paramater of up to 19.85Million where I used Context of 35


In [17]:
class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        # Creating a sequential container of layers
        
        self.layers = torch.nn.Sequential( 
            # First Hidden Layer with 2048 neurons
            torch.nn.Linear(input_size, 2048),   
            torch.nn.GELU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(0.3),
            
            # Second Hidden layer with 1024 Neurons
            torch.nn.Linear(2048, 1024),
            torch.nn.GELU(),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Dropout(0.3),
            
            # Third Hidden Layer with 2048 neurons
            torch.nn.Linear(1024, 2048),
            torch.nn.GELU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(0.3),
            
            # Fourth Hidden Layer with 2048 Neurons
            torch.nn.Linear(2048, 2048),
            torch.nn.GELU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(0.3),
            
            # Fifth hidden layer with 2048 neurons
            torch.nn.Linear(2048, 2048),
            torch.nn.GELU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(0.3),
            
            # Sixth hidden layer with 1024 neurons
            torch.nn.Linear(2048, 1024),
            torch.nn.GELU(),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Dropout(0.3),
            
            # seventh Hidden layer 1024 Neurons
            torch.nn.Linear(1024, 1024),
            torch.nn.GELU(),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Dropout(0.3),
            
            
            torch.nn.Linear(1024, output_size)   
        )
        
         
    # Weight Initialization 
    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_normal_(module.weight)  
            if module.bias is not None:
                torch.nn.init.constant_(module.bias, 0)
        elif isinstance(module, torch.nn.BatchNorm1d):
            torch.nn.init.constant_(module.weight, 1)
            torch.nn.init.constant_(module.bias, 0)
           
    def forward(self, x):
        return self.layers(x)


# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [18]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)

summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

----------------------------------------------------------------------------------------------------

Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)


0_Linear                [1988, 2048]         [8192, 2048]             4,073.47                 4.07

1_GELU                             -         [8192, 2048]                    -                    -

2_BatchNorm1d                 [2048]         [8192, 2048]                 4.10                 0.00

3_Dropout                          -         [8192, 2048]                    -                    -

4_Linear                [2048, 1024]         [8192, 1024]             2,098.18                 2.10

5_GELU                             -         [8192, 1024]                    -                    -

6_BatchNorm1d                 [1024]         [8192, 1024]                 2.05                 0.00

7_Dropout                          -         [8192, 1024]                    -           

#### Given the CrossEntropyLoss for multi-class classification. My team and I experimented with various optimizers and found that the AdamW optimizer, with its integrated weight decay for regularization, delivered the best results for our multi-class classification task. We utilized the ReduceLROnPlateau scheduler to dynamically adjust the learning rate based on validation loss, which helped enhance model performance. Although the CosineAnnealingLR scheduler also showed promise in improving efficiency, I chose ReduceLROnPlateau as Learning Rate Scheduler.

In [19]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

# optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr']) #Defining Optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr=config['init_lr'], weight_decay=1e-3) 
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStep, CosineAnnealing, CosineAnnealingWithWarmRestarts, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min', 
    patience=3, 
    verbose=False, 
    factor=0.1
)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max= 7500,eta_min=0)
# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.

  warnings.warn(


# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

I did not modify the cells of Train and Validation 

In [20]:
torch.cuda.empty_cache()
gc.collect()

0

In [21]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [22]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [23]:
wandb.login(key="889736f968fd287026464ea18bead25ea75991d4") #API Key is in your wandb account, under settings (wandb.ai/settings)
                

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.

wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

I did 4 runs after seeing my model try to perform well I had to resume it every time until it gave me the High cutoff, so below I included the code to reload the best model since I was saving the best model at every epoch where this approach ensures that I can reload the best model from any desired epoch, offering flexibility in refining the model's accuracy by continuing from the most effective training state.

In [24]:
# Create your wandb run
run = wandb.init(
    name    = "ange-hw1p1-13/09-2024", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    #reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    id     = "vbuxod3v", ### Insert specific run id here if you want to resume a previous run
    resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)
reload_m = wandb.restore('new_models_12/best_model_GELU_15_13.pth')
model.load_state_dict(torch.load(reload_m.name))

wandb: Currently logged in as: aizabayo (aizabayo-carnegie-mellon-university). Use `wandb login --relogin` to force relogin

wandb: Tracking run with wandb version 0.18.1

wandb: Run data is saved locally in /kaggle/working/wandb/run-20240925_204258-vbuxod3v

wandb: Run `wandb offline` to turn off syncing.

wandb: Resuming run ange-hw1p1-13/09-2024

wandb: ⭐️ View project at https://wandb.ai/aizabayo-carnegie-mellon-university/hw1p2

wandb: 🚀 View run at https://wandb.ai/aizabayo-carnegie-mellon-university/hw1p2/runs/vbuxod3v

/tmp/ipykernel_22/2699587269.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. 

<All keys matched successfully>

In [25]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/kaggle/working/wandb/run-20240925_204258-vbuxod3v/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

#### In the training loop, I implemented added the code to save the best model after each epoch, logging details on Weights & Biases (wandb) in its own folder

#### Initially, I conducted a 30-epoch run which achieved a validation accuracy of 85.47%. Encouraged by the model's performance and absence of overfitting, I resumed training for an additional 15 epochs, reaching a validation accuracy of 85.84% with a training accuracy of 84.6%. Although the improvement was modest, I proceeded with another 15-epoch session. However, the increase in validation accuracy was minimal, reaching only 86.01%. To ensure thorough evaluation, I extended the training by five more epochs, as shown in this notebook below which is the one I turned in.



In [26]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
best_acc = 0
for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), f'best_model_epoch_{epoch+1}.pth')
        print(f"Best model saved with validation accuracy: {val_acc*100:.04f}%")
        
        m_path = os.path.join(wandb.run.dir, "new_models_13", f"best_model_GELU_16_{epoch+1}.pth")

        # Create the necessary directories if they do not exist
        os.makedirs(os.path.dirname(m_path), exist_ok=True)

        # Save the model
        torch.save(model.state_dict(), m_path)
        wandb.save(m_path)





Epoch 1/5


Train:   0%|          | 0/4406 [00:00<?, ?it/s]

Val:   0%|          | 0/236 [00:00<?, ?it/s]

	Train Acc 84.9180%	Train Loss 0.4372	 Learning Rate 0.0010000

	Val Acc 86.0419%	Val Loss 0.4091

Best model saved with validation accuracy: 86.0419%


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


...

Val:   0%|          | 0/236 [00:00<?, ?it/s]

	Train Acc 84.9682%	Train Loss 0.4352	 Learning Rate 0.0010000

	Val Acc 86.0315%	Val Loss 0.4091



Epoch 5/5


Train:   0%|          | 0/4406 [00:00<?, ?it/s]

Val:   0%|          | 0/236 [00:00<?, ?it/s]

	Train Acc 84.9873%	Train Loss 0.4348	 Learning Rate 0.0010000

	Val Acc 86.1182%	Val Loss 0.4084

Best model saved with validation accuracy: 86.1182%


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [27]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            #predicted_phonemes = 

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            # TODO
            predicted_phonemes=torch.argmax(logits, dim= 1)
            test_predictions.extend(predicted_phonemes) 

    return test_predictions

In [28]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
predictions = test(model, test_loader)

  0%|          | 0/237 [00:00<?, ?it/s]

#### In the AudioDataset class, I converted each phoneme into a unique index to facilitate numerical processing required by machine learning models. Before generating the CSV file for predictions, it was necessary to map these unique indices back to their corresponding phonemes, as these are the actual targets of our predictions. Once this mapping was completed, I proceeded to create the CSV file, detailing each prediction with its respective phoneme label

In [32]:
### Create CSV file with predictions
with open("./trial_submission2-3.csv", "w") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        phonem = PHONEMES[predictions[i]]
        f.write("{},{}\n".format(i, phonem))

In [33]:
### Finish your wandb run
run.finish()

wandb:                                                                                

wandb: 

wandb: Run history:

wandb:         lr ▁▁▁▁▁

wandb:  train_acc ▁▃▄▆█

wandb: train_loss █▅▄▂▁

wandb:    val_acc ▃▁▂▂█

wandb: valid_loss ▃█▅▃▁

wandb: 

wandb: Run summary:

wandb:         lr 0.001

wandb:  train_acc 84.98734

wandb: train_loss 0.4348

wandb:    val_acc 86.11823

wandb: valid_loss 0.40845

wandb: 

wandb: 🚀 View run ange-hw1p1-13/09-2024 at: https://wandb.ai/aizabayo-carnegie-mellon-university/hw1p2/runs/vbuxod3v

wandb: ⭐️ View project at: https://wandb.ai/aizabayo-carnegie-mellon-university/hw1p2

wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 3 other file(s)

wandb: Find logs at: ./wandb/run-20240925_204258-vbuxod3v/logs


#### Since I was testing many hyperparameter I commented the code for submitting the prediction csv file every time I run. So that I can do it manually.

In [34]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
# !kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle